In [1]:
require "nyaplot"
require "nyaplot3d"
require "json"
require "daru"
require "statsample"

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\",\"THREE\":\"http://cdnjs.cloudflare.com/ajax/libs/three.js/r66/three.min\",\"Elegans\":\"https://cdn.rawgit.com/domitry/elegans/d81a728f62edaeeb67261516a272438fb39fa80a/release/elegans\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"},\"THREE\":{\"exports\":\"THREE\"},\"Elegans\":{\"exports\":\"Elegans\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');require(['THREE'], function(THREE){window['THREE']=THREE;console.log('finished loading THREE');require(['Elegans'], function(Eleg

true

In [2]:
sensor_names = {
  "32" => :ambient_temp,
  "96" => :systemboard_temp,
  "160" => :cpu1_temp,
  "224" => :cpu2_temp,
  "288" => :dimm_1a_temp,
  "1824" => :main_12v,
  "1888" => :main_5v,
  "1952" => :main_3_3v,
  "2272" => :cpu1_1_8v,
  "2336" => :cpu2_1_8v,
  "2400" => :fan1_sys,
  "2464" => :fan2_sys,
  "2528" => :fan3_sys,
  "2592" => :fan4_sys,
  "2656" => :fan5_sys,
  "3296" => :total_power
}

{"32"=>:ambient_temp, "96"=>:systemboard_temp, "160"=>:cpu1_temp, "224"=>:cpu2_temp, "288"=>:dimm_1a_temp, "1824"=>:main_12v, "1888"=>:main_5v, "1952"=>:main_3_3v, "2272"=>:cpu1_1_8v, "2336"=>:cpu2_1_8v, "2400"=>:fan1_sys, "2464"=>:fan2_sys, "2528"=>:fan3_sys, "2592"=>:fan4_sys, "2656"=>:fan5_sys, "3296"=>:total_power}

In [3]:
response = JSON.load(File.read("6358153b-064b-5b11-bfc6-2a45cb0dc42a-7d30m.json"))
data = response["result"]["data"][0]
time_idx = data["columns"].index { |i| i["name"] == "time" }
sensor_no_idx = data["columns"].index { |i| i["name"] == "sensor_no" }
value_idx = data["columns"].index { |i| i["name"] == "mean" }

h = sensor_names.values.each_with_object({time: []}) {|name, h| h[name] = []}
data["rows"].chunk_while { |i, j|
  i[time_idx] == j[time_idx]
}.each do |rows|
  h[:time].push(rows[0][time_idx] / 1000.0 / 3600.0 / 24.0) # days since epoch
  rows.each { |row|
    no = row[sensor_no_idx]
    name = sensor_names[no]
    if name =~ /_temp\z/
      value = row[value_idx] / 1000.0 - 273.15  # millikelvin to celsius
    else
      value = row[value_idx] / 1000.0
    end
    h[name].push(value)
  }
end
df = Daru::DataFrame.new(h, order: [:time] + sensor_names.values, dtype: :nmatrix)

,time,ambient_temp,systemboard_temp,cpu1_temp,cpu2_temp,dimm_1a_temp,main_12v,main_5v,main_3_3v,cpu1_1_8v,cpu2_1_8v,fan1_sys,fan2_sys,fan3_sys,fan4_sys,fan5_sys,total_power
0,16839.333333333332,18.887500000000045,23.025000000000034,32.200000000000045,32.700000000000045,22.0,12.0,5.13,3.4,1.7845,1.78,6102.0,6153.0,5775.0,6120.0,6006.0,92.15
1,16839.354166666668,18.574074074074133,23.0,32.08641975308643,32.382716049382736,22.0,12.0,5.13,3.4,1.7858024691358025,1.78,6099.259259259259,6154.074074074074,5783.7037037037035,6120.0,6004.444444444444,92.0
2,16839.375,18.128048780487802,22.341463414634177,31.280487804878078,32.21951219512198,21.024390243902474,12.0,5.13,3.4,1.7860975609756096,1.78,6103.90243902439,6130.243902439024,5777.5609756097565,6120.0,6007.317073170731,92.09756097560975
3,16839.395833333332,17.780487804878078,22.0,30.81707317073176,31.890243902439067,21.0,12.0,5.13,3.4,1.7870731707317074,1.78,6095.121951219512,6140.487804878048,5786.341463414635,6120.0,6004.39024390244,92.14634146341463
4,16839.416666666668,17.481250000000045,21.98750000000001,30.61250000000001,31.4375,20.61250000000001,12.0,5.13,3.4,1.785875,1.78,6111.0,6154.5,5778.0,6120.0,6003.0,93.35
5,16839.4375,17.419753086419803,21.51851851851859,30.62962962962962,31.407407407407447,20.0,12.0,5.13,3.4,1.7883950617283952,1.78,6102.222222222222,6134.814814814815,5779.259259259259,6120.0,6000.0,94.51851851851852
6,16839.458333333332,17.256250000000023,21.11250000000001,30.275000000000034,31.150000000000034,20.0,12.0,5.13,3.4,1.788375,1.78,6102.0,6124.5,5779.5,6120.0,6003.0,92.0
7,16839.479166666668,16.925925925925924,21.0,30.074074074074133,31.160493827160508,20.0,12.0,5.13,3.4,1.7874074074074073,1.78,6105.185185185185,6122.962962962963,5785.185185185185,6120.0,6013.333333333333,92.04938271604938
8,16839.5,16.89634146341467,21.0,30.0609756097561,31.012195121951265,20.0,12.0,5.13,3.4,1.788048780487805,1.78,6105.365853658536,6133.170731707317,5805.365853658536,6120.0,6005.8536585365855,92.04878048780488
9,16839.520833333332,16.432926829268297,20.81707317073176,30.012195121951265,30.756097560975604,19.439024390243958,12.0,5.13,3.4,1.788780487804878,1.78,6106.829268292683,6137.5609756097565,5768.780487804878,6120.0,6002.926829268293,92.04878048780488


In [4]:
df.describe

,ambient_temp,cpu1_1_8v,cpu1_temp,cpu2_1_8v,cpu2_temp,dimm_1a_temp,fan1_sys,fan2_sys,fan3_sys,fan4_sys,fan5_sys,main_12v,main_3_3v,main_5v,systemboard_temp,time,total_power
count,336,336,336,336,336,336,336,336,336,336,336,336,336,336,336,336,336
mean,18.57342679223287,1.7855751796405162,32.85246269942335,1.779999999999985,32.83955662587552,21.632263084901677,6143.9142686887435,6193.032706039083,5827.531923377075,6163.384803688228,6043.446333162844,11.99904466599672,3.399999999999995,5.1300000000000425,22.737816662123898,16842.822916666668,92.42382928769287
std,3.3887922388812286,0.003101598746090587,2.722765225222846,1.5121552212493516e-14,2.6107806876371553,3.3676906086024023,311.70079264333015,333.7547281917533,290.31139008852745,333.2609392343002,292.3731930455626,0.007186675210921292,4.892266892277314e-15,4.2696147423511106e-14,3.43324361815764,2.0237307407634826,1.557162061419941
min,10.493243243243285,1.78,30.0,1.78,30.0,13.0,6083.333333333333,6121.5,5766.153846153846,6120.0,6000.0,11.94,3.4,5.13,14.31578947368422,16839.333333333332,92.0
max,30.986486486486513,1.79,44.3648648648649,1.78,45.0,33.21621621621625,9360.0,9702.162162162162,8938.378378378378,9715.135135135135,9201.081081081082,12.0,3.4,5.13,34.98648648648651,16846.3125,107.89189189189189


In [5]:
cm = Statsample::Bivariate.correlation_matrix(df)
open("correlation_matrix.csv", "w") do |f|
  columns = df.vectors.to_a.map(&:to_s)
  f.puts("," + columns.join(","))
  cm.rows.each_with_index do |row, i|
    f.puts(columns[i] + "," + row.join(","))
  end
end
cm

Matrix[[1.0, 0.11556339584553221, 0.12401486019457411, 0.24702517504153518, 0.16683727016609567, 0.13980834450074114, -0.023039343852868363, 6.00113527416051e-13, -6.001135274160511e-13, -0.18420521035408327, -6.00113527416051e-13, 0.020752697200008087, 0.0422990984379249, 0.025767684865026962, 0.022861669775908908, 0.023022696171999136, 0.01951333164311044], [0.11556339584553221, 1.0, 0.9950900318718721, 0.9185751977823463, 0.9298901571440621, 0.9965801707902165, -0.423287289524894, -1.4426193090546984e-15, 1.7027358329781153e-15, -0.906204496174289, 1.5862991680367864e-15, 0.4088898319432894, 0.4392719241687862, 0.41185809685608027, 0.42208356000678027, 0.41273484043632214, 0.3569234971521401], [0.12401486019457411, 0.9950900318718721, 1.0, 0.9197193539301823, 0.9317770222528537, 0.9958379746753175, -0.41190815230410854, -2.1590444543918806e-15, 2.659830609657322e-15, -0.9218420579757518, 2.717855960234485e-15, 0.39688634133871115, 0.4296375048837063, 0.4006089281897102, 0.4104863067

In [6]:
plot = Nyaplot::Plot.new
plot.add_with_df(df, :line, :time, :cpu1_temp).configure do |d|
  d.title("CPU1")
  d.color("#dd4444")
end
plot.add_with_df(df, :line, :time, :cpu2_temp).configure do |d|
  d.title("CPU2")
  d.color("#ddaa44")
end
plot.add_with_df(df, :line, :time, :systemboard_temp).configure do |d|
  d.title("Systemboard")
  d.color("#44bb44")
end
plot.add_with_df(df, :line, :time, :ambient_temp).configure do |d|
  d.title("Ambient")
end
plot.x_label("Time (days since epoch)")
plot.y_label("Temperature (℃)")
plot.legend(true)
plot.show

#<Nyaplot::Frame:0x007f953ef52640 @properties={:panes=>[#<Nyaplot::Plot:0x007f953ef69110 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f953ef68f08 @properties={:type=>:line, :options=>{:x=>:time, :y=>:cpu1_temp, :title=>"CPU1", :color=>"#dd4444"}, :data=>"9ca2e756-a231-4e5d-b3db-384882ffdc29"}, @xrange=[16839.333333333332, 16846.3125], @yrange=[30.0, 44.3648648648649]>, #<Nyaplot::Diagram:0x007f953ef68440 @properties={:type=>:line, :options=>{:x=>:time, :y=>:cpu2_temp, :title=>"CPU2", :color=>"#ddaa44"}, :data=>"9ca2e756-a231-4e5d-b3db-384882ffdc29"}, @xrange=[16839.333333333332, 16846.3125], @yrange=[30.0, 45.0]>, #<Nyaplot::Diagram:0x007f953ef539f0 @properties={:type=>:line, :options=>{:x=>:time, :y=>:systemboard_temp, :title=>"Systemboard", :color=>"#44bb44"}, :data=>"9ca2e756-a231-4e5d-b3db-384882ffdc29"}, @xrange=[16839.333333333332, 16846.3125], @yrange=[14.31578947368422, 34.98648648648651]>, #<Nyaplot::Diagram:0x007f953ef53018 @properties={:type=>:line, :options=>{:x=>:time, :y=>:ambient_temp, :title=>"Ambient"}, :data=>"9ca2e756-a231-4e5d-b3db-384882ffdc29"}, @xrange=[16839.333333333332, 16846.3125], @yrange=[10.493243243243285, 30.986486486486513]>], :options=>{:x_label=>"Time (days since epoch)", :y_label=>"Temperature (℃)", :legend=>true, :zoom=>true, :width=>800, :xrange=>[16839.333333333332, 16846.3125], :yrange=>[10.493243243243285, 45.0]}}>], :data=>{"9ca2e756-a231-4e5d-b3db-384882ffdc29"=>
#<Daru::DataFrame:70139470564160 @name = 9ca2e756-a231-4e5d-b3db-384882ffdc29 @size = 336>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 16839.3333 18.8875000 23.0250000 32.2000000 32.7000000       22.0       12.0       5.13        3.4     1.7845       1.78     6102.0     6153.0     5775.0     6120.0     6006.0      92.15 
         1 16839.3541 18.5740740       23.0 32.0864197 32.3827160       22.0       12.0       5.13        3.4 1.78580246       1.78 6099.25925 6154.07407 5783.70370     6120.0 6004.44444       92.0 
         2  16839.375 18.1280487 22.3414634 31.2804878 32.2195121 21.0243902       12.0       5.13        3.4 1.78609756       1.78 6103.90243 6130.24390 5777.56097     6120.0 6007.31707 92.0975609 
         3 16839.3958 17.7804878       22.0 30.8170731 31.8902439       21.0       12.0       5.13        3.4 1.78707317       1.78 6095.12195 6140.48780 5786.34146     6120.0 6004.39024 92.1463414 
         4 16839.4166 17.4812500 21.9875000 30.6125000    31.4375 20.6125000       12.0       5.13        3.4   1.785875       1.78     6111.0     6154.5     5778.0     6120.0     6003.0      93.35 
         5 16839.4375 17.4197530 21.5185185 30.6296296 31.4074074       20.0       12.0       5.13        3.4 1.78839506       1.78 6102.22222 6134.81481 5779.25925     6120.0     6000.0 94.5185185 
         6 16839.4583 17.2562500 21.1125000 30.2750000 31.1500000       20.0       12.0       5.13        3.4   1.788375       1.78     6102.0     6124.5     5779.5     6120.0     6003.0       92.0 
         7 16839.4791 16.9259259       21.0 30.0740740 31.1604938       20.0       12.0       5.13        3.4 1.78740740       1.78 6105.18518 6122.96296 5785.18518     6120.0 6013.33333 92.0493827 
         8    16839.5 16.8963414       21.0 30.0609756 31.0121951       20.0       12.0       5.13        3.4 1.78804878       1.78 6105.36585 6133.17073 5805.36585     6120.0 6005.85365 92.0487804 
         9 16839.5208 16.4329268 20.8170731 30.0121951 30.7560975 19.4390243       12.0       5.13        3.4 1.78878048       1.78 6106.82926 6137.56097 5768.78048     6120.0 6002.92682 92.0487804 
        10 16839.5416 15.9187500 20.0125000       30.0 30.4625000       19.0       12.0       5.13        3.4   1.788625       1.78     6108.0     6127.5     5784.0     6120.0     6010.5      92.05 
        11 16839.5625     15.875       20.0       30.0 30.2738095       19.0       12.0 

In [7]:
plot = Nyaplot::Plot.new
plot.add_with_df(df, :scatter, :cpu1_temp, :ambient_temp)
plot.x_label("CPU1 temperature (℃)")
plot.y_label("Ambient temperature (℃)")
plot.show

#<Nyaplot::Frame:0x007f953ebfbed8 @properties={:panes=>[#<Nyaplot::Plot:0x007f953ebf09c0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f953ebf0808 @properties={:type=>:scatter, :options=>{:x=>:cpu1_temp, :y=>:ambient_temp}, :data=>"9ca2e756-a231-4e5d-b3db-384882ffdc29"}, @xrange=[30.0, 44.3648648648649], @yrange=[10.493243243243285, 30.986486486486513]>], :options=>{:x_label=>"CPU1 temperature (℃)", :y_label=>"Ambient temperature (℃)", :zoom=>true, :width=>700, :xrange=>[30.0, 44.3648648648649], :yrange=>[10.493243243243285, 30.986486486486513]}}>], :data=>{"9ca2e756-a231-4e5d-b3db-384882ffdc29"=>
#<Daru::DataFrame:70139470564160 @name = 9ca2e756-a231-4e5d-b3db-384882ffdc29 @size = 336>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 16839.3333 18.8875000 23.0250000 32.2000000 32.7000000       22.0       12.0       5.13        3.4     1.7845       1.78     6102.0     6153.0     5775.0     6120.0     6006.0      92.15 
         1 16839.3541 18.5740740       23.0 32.0864197 32.3827160       22.0       12.0       5.13        3.4 1.78580246       1.78 6099.25925 6154.07407 5783.70370     6120.0 6004.44444       92.0 
         2  16839.375 18.1280487 22.3414634 31.2804878 32.2195121 21.0243902       12.0       5.13        3.4 1.78609756       1.78 6103.90243 6130.24390 5777.56097     6120.0 6007.31707 92.0975609 
         3 16839.3958 17.7804878       22.0 30.8170731 31.8902439       21.0       12.0       5.13        3.4 1.78707317       1.78 6095.12195 6140.48780 5786.34146     6120.0 6004.39024 92.1463414 
         4 16839.4166 17.4812500 21.9875000 30.6125000    31.4375 20.6125000       12.0       5.13        3.4   1.785875       1.78     6111.0     6154.5     5778.0     6120.0     6003.0      93.35 
         5 16839.4375 17.4197530 21.5185185 30.6296296 31.4074074       20.0       12.0       5.13        3.4 1.78839506       1.78 6102.22222 6134.81481 5779.25925     6120.0     6000.0 94.5185185 
         6 16839.4583 17.2562500 21.1125000 30.2750000 31.1500000       20.0       12.0       5.13        3.4   1.788375       1.78     6102.0     6124.5     5779.5     6120.0     6003.0       92.0 
         7 16839.4791 16.9259259       21.0 30.0740740 31.1604938       20.0       12.0       5.13        3.4 1.78740740       1.78 6105.18518 6122.96296 5785.18518     6120.0 6013.33333 92.0493827 
         8    16839.5 16.8963414       21.0 30.0609756 31.0121951       20.0       12.0       5.13        3.4 1.78804878       1.78 6105.36585 6133.17073 5805.36585     6120.0 6005.85365 92.0487804 
         9 16839.5208 16.4329268 20.8170731 30.0121951 30.7560975 19.4390243       12.0       5.13        3.4 1.78878048       1.78 6106.82926 6137.56097 5768.78048     6120.0 6002.92682 92.0487804 
        10 16839.5416 15.9187500 20.0125000       30.0 30.4625000       19.0       12.0       5.13        3.4   1.788625       1.78     6108.0     6127.5     5784.0     6120.0     6010.5      92.05 
        11 16839.5625     15.875       20.0       30.0 30.2738095       19.0       12.0       5.13        3.4 1.78880952       1.78 6104.28571 6134.28571 5774.28571     6120.0 6007.14285       92.0 
        12 16839.5833 16.3012048 20.2530120       30.0 30.6024096       19.0       12.0       5.13        3.4 1.78867469       1.78 6105.54216 6128.67469 5780.24096     6120.0 6010.12048       92.0 
        13 16839.6041 16.4036144 20.9879518       30.0 30.9518072 19.4457831       12.0       5.13        3.4 1.78771084       1.78 6099.75903 6128.67469 5773.01204     6120.0 6001.44578 92.0481927 
        14  16839.625 16.3475609 20.4634146 30.0121951 30.4756097       19.0       12.0       5.13        3.4 1.78902439       1.78 6102.43902 6130.24390 5787.80487     6120.0 6005.85365 92.0487804 
       ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        

In [8]:
filtered_df = df.where(df[:cpu1_temp].gt(30.0))
plot = Nyaplot::Plot.new
plot.add_with_df(filtered_df, :scatter, :cpu1_temp, :ambient_temp)
plot.x_label("CPU1 temperature (℃)")
plot.y_label("Ambient temperature (℃)")
plot.show

#<Nyaplot::Frame:0x007f953f1406f0 @properties={:panes=>[#<Nyaplot::Plot:0x007f953f1411b8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f953f141000 @properties={:type=>:scatter, :options=>{:x=>:cpu1_temp, :y=>:ambient_temp}, :data=>"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"}, @xrange=[30.012195121951265, 44.3648648648649], @yrange=[15.287671232876733, 30.986486486486513]>], :options=>{:x_label=>"CPU1 temperature (℃)", :y_label=>"Ambient temperature (℃)", :zoom=>true, :width=>700, :xrange=>[30.012195121951265, 44.3648648648649], :yrange=>[15.287671232876733, 30.986486486486513]}}>], :data=>{"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"=>
#<Daru::DataFrame:70139492567880 @name = 8f45396d-a66c-4e4b-ac80-f4a63b36a9f8 @size = 266>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 16839.3333 18.8875000 23.0250000 32.2000000 32.7000000       22.0       12.0       5.13        3.4     1.7845       1.78     6102.0     6153.0     5775.0     6120.0     6006.0      92.15 
         1 16839.3541 18.5740740       23.0 32.0864197 32.3827160       22.0       12.0       5.13        3.4 1.78580246       1.78 6099.25925 6154.07407 5783.70370     6120.0 6004.44444       92.0 
         2  16839.375 18.1280487 22.3414634 31.2804878 32.2195121 21.0243902       12.0       5.13        3.4 1.78609756       1.78 6103.90243 6130.24390 5777.56097     6120.0 6007.31707 92.0975609 
         3 16839.3958 17.7804878       22.0 30.8170731 31.8902439       21.0       12.0       5.13        3.4 1.78707317       1.78 6095.12195 6140.48780 5786.34146     6120.0 6004.39024 92.1463414 
         4 16839.4166 17.4812500 21.9875000 30.6125000    31.4375 20.6125000       12.0       5.13        3.4   1.785875       1.78     6111.0     6154.5     5778.0     6120.0     6003.0      93.35 
         5 16839.4375 17.4197530 21.5185185 30.6296296 31.4074074       20.0       12.0       5.13        3.4 1.78839506       1.78 6102.22222 6134.81481 5779.25925     6120.0     6000.0 94.5185185 
         6 16839.4583 17.2562500 21.1125000 30.2750000 31.1500000       20.0       12.0       5.13        3.4   1.788375       1.78     6102.0     6124.5     5779.5     6120.0     6003.0       92.0 
         7 16839.4791 16.9259259       21.0 30.0740740 31.1604938       20.0       12.0       5.13        3.4 1.78740740       1.78 6105.18518 6122.96296 5785.18518     6120.0 6013.33333 92.0493827 
         8    16839.5 16.8963414       21.0 30.0609756 31.0121951       20.0       12.0       5.13        3.4 1.78804878       1.78 6105.36585 6133.17073 5805.36585     6120.0 6005.85365 92.0487804 
         9 16839.5208 16.4329268 20.8170731 30.0121951 30.7560975 19.4390243       12.0       5.13        3.4 1.78878048       1.78 6106.82926 6137.56097 5768.78048     6120.0 6002.92682 92.0487804 
        14  16839.625 16.3475609 20.4634146 30.0121951 30.4756097       19.0       12.0       5.13        3.4 1.78902439       1.78 6102.43902 6130.24390 5787.80487     6120.0 6005.85365 92.0487804 
        18 16839.7083 16.6582278       21.0 30.0759493 30.5696202       20.0       12.0       5.13        3.4 1.78810126       1.78 6100.25316 6127.59493 5784.30379     6120.0 6007.59493 92.0506329 
        19 16839.7291 17.0379746 21.1772151 30.4936708 30.9620253       20.0       12.0       5.13        3.4 1.78772151       1.78 6095.69620 6123.03797 5772.15189     6120.0 6006.07594 92.0506329 
        20   16839.75 17.3961038 21.9740259 30.6493506 31.3766233 20.1038961       12.0       5.13        3.4 1.78844155       1.78 6101.29870 6121.55844 5777.14285     6120.0 6007.79220 92.0519480 
        21 16839.7708 17.7266666       22.0 30.8400000 31.7333333       21.0       12.0       5.13        3.4 1.78733333       1.78     6104.0     6132.8     5779.2     6120.0     6004.8 92.0533333 
       ...        ...        ...        ...        ...        ...        ...        ...  

In [9]:
sr = Statsample::Regression.simple(filtered_df[:cpu1_temp], filtered_df[:ambient_temp])
cpu1_temp_min = filtered_df[:cpu1_temp].min
cpu1_temp_max = filtered_df[:cpu1_temp].max
df2 = Daru::DataFrame.new(
  cpu1_temp_min.step(cpu1_temp_max, (cpu1_temp_max - cpu1_temp_min) / 10.0).map { |x|
    { x: x, y: sr.a + sr.b * x }
  }
)
plot = Nyaplot::Plot.new
plot.add_with_df(filtered_df, :scatter, :cpu1_temp, :ambient_temp).configure do |d|
  d.title("Sample")
end
plot.add_with_df(df2, :line, :x, :y).configure do |d|
  d.color("#44bb44")
  d.title(format("%.3f + %.3fx", sr.a, sr.b))
end
plot.x_label("CPU1 temperature (℃)")
plot.y_label("Ambient temperature (℃)")
plot.legend(true)
plot.show


#<Nyaplot::Frame:0x007f953f04a700 @properties={:panes=>[#<Nyaplot::Plot:0x007f953f04bd08 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f953f04bb50 @properties={:type=>:scatter, :options=>{:x=>:cpu1_temp, :y=>:ambient_temp, :title=>"Sample"}, :data=>"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"}, @xrange=[30.012195121951265, 44.3648648648649], @yrange=[15.287671232876733, 30.986486486486513]>, #<Nyaplot::Diagram:0x007f953f04b1f0 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :color=>"#44bb44", :title=>"-10.871 + 0.913x"}, :data=>"691320fe-bb65-44fa-84c7-a84c83d14e0d"}, @xrange=[30.012195121951265, 44.3648648648649], @yrange=[16.541943637969787, 29.65167584226349]>], :options=>{:x_label=>"CPU1 temperature (℃)", :y_label=>"Ambient temperature (℃)", :legend=>true, :zoom=>true, :width=>800, :xrange=>[30.012195121951265, 44.3648648648649], :yrange=>[15.287671232876733, 30.986486486486513]}}>], :data=>{"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"=>
#<Daru::DataFrame:70139492567880 @name = 8f45396d-a66c-4e4b-ac80-f4a63b36a9f8 @size = 266>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 16839.3333 18.8875000 23.0250000 32.2000000 32.7000000       22.0       12.0       5.13        3.4     1.7845       1.78     6102.0     6153.0     5775.0     6120.0     6006.0      92.15 
         1 16839.3541 18.5740740       23.0 32.0864197 32.3827160       22.0       12.0       5.13        3.4 1.78580246       1.78 6099.25925 6154.07407 5783.70370     6120.0 6004.44444       92.0 
         2  16839.375 18.1280487 22.3414634 31.2804878 32.2195121 21.0243902       12.0       5.13        3.4 1.78609756       1.78 6103.90243 6130.24390 5777.56097     6120.0 6007.31707 92.0975609 
         3 16839.3958 17.7804878       22.0 30.8170731 31.8902439       21.0       12.0       5.13        3.4 1.78707317       1.78 6095.12195 6140.48780 5786.34146     6120.0 6004.39024 92.1463414 
         4 16839.4166 17.4812500 21.9875000 30.6125000    31.4375 20.6125000       12.0       5.13        3.4   1.785875       1.78     6111.0     6154.5     5778.0     6120.0     6003.0      93.35 
         5 16839.4375 17.4197530 21.5185185 30.6296296 31.4074074       20.0       12.0       5.13        3.4 1.78839506       1.78 6102.22222 6134.81481 5779.25925     6120.0     6000.0 94.5185185 
         6 16839.4583 17.2562500 21.1125000 30.2750000 31.1500000       20.0       12.0       5.13        3.4   1.788375       1.78     6102.0     6124.5     5779.5     6120.0     6003.0       92.0 
         7 16839.4791 16.9259259       21.0 30.0740740 31.1604938       20.0       12.0       5.13        3.4 1.78740740       1.78 6105.18518 6122.96296 5785.18518     6120.0 6013.33333 92.0493827 
         8    16839.5 16.8963414       21.0 30.0609756 31.0121951       20.0       12.0       5.13        3.4 1.78804878       1.78 6105.36585 6133.17073 5805.36585     6120.0 6005.85365 92.0487804 
         9 16839.5208 16.4329268 20.8170731 30.0121951 30.7560975 19.4390243       12.0       5.13        3.4 1.78878048       1.78 6106.82926 6137.56097 5768.78048     6120.0 6002.92682 92.0487804 
        14  16839.625 16.3475609 20.4634146 30.0121951 30.4756097       19.0       12.0       5.13        3.4 1.78902439       1.78 6102.43902 6130.24390 5787.80487     6120.0 6005.85365 92.0487804 
        18 16839.7083 16.6582278       21.0 30.0759493 30.5696202       20.0       12.0       5.13        3.4 1.78810126       1.78 6100.25316 6127.59493 5784.30379     6120.0 6007.59493 92.0506329 
        19 16839.7291 17.0379746 21.1772151 30.4936708 30.9620253       20.0       12.0       5.13        3.4 1.78772151       1.78 6095.69620 6123.03797 5772.15189     6120.0 6006.07594 92.0506329 
        20   16839.75 17.3961038 21.9740259 30.6493506 31.3766233 20.1038961       12.0       5.13        3.4 1.78844155       1.78 6101.29870 6121.55844 5777.14285     6

In [10]:
filtered_df.where(filtered_df[:cpu1_temp].gt(31.0) & filtered_df[:ambient_temp].lt(16))

,time,ambient_temp,systemboard_temp,cpu1_temp,cpu2_temp,dimm_1a_temp,main_12v,main_5v,main_3_3v,cpu1_1_8v,cpu2_1_8v,fan1_sys,fan2_sys,fan3_sys,fan4_sys,fan5_sys,total_power
330,16846.208333333332,15.526315789473756,21.15789473684208,31.75,31.934210526315837,20.013157894736878,12.0,5.13,3.4,1.786315789473684,1.78,6102.631578947368,6162.631578947368,5775.78947368421,6120.0,6011.0526315789475,92.10526315789474


In [11]:
plot = Nyaplot::Plot.new
plot.add_with_df(filtered_df, :line, :time, :cpu1_1_8v)
plot.x_label("Time (days since epoch)")
plot.y_label("CPU1 Voltage (W)")
plot.show

#<Nyaplot::Frame:0x007f953ee7c5b8 @properties={:panes=>[#<Nyaplot::Plot:0x007f953ee7d058 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f953ee7cea0 @properties={:type=>:line, :options=>{:x=>:time, :y=>:cpu1_1_8v}, :data=>"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"}, @xrange=[16839.333333333332, 16846.3125], @yrange=[1.78, 1.7895890410958906]>], :options=>{:x_label=>"Time (days since epoch)", :y_label=>"CPU1 Voltage (W)", :zoom=>true, :width=>700, :xrange=>[16839.333333333332, 16846.3125], :yrange=>[1.78, 1.7895890410958906]}}>], :data=>{"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"=>
#<Daru::DataFrame:70139492567880 @name = 8f45396d-a66c-4e4b-ac80-f4a63b36a9f8 @size = 266>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 16839.3333 18.8875000 23.0250000 32.2000000 32.7000000       22.0       12.0       5.13        3.4     1.7845       1.78     6102.0     6153.0     5775.0     6120.0     6006.0      92.15 
         1 16839.3541 18.5740740       23.0 32.0864197 32.3827160       22.0       12.0       5.13        3.4 1.78580246       1.78 6099.25925 6154.07407 5783.70370     6120.0 6004.44444       92.0 
         2  16839.375 18.1280487 22.3414634 31.2804878 32.2195121 21.0243902       12.0       5.13        3.4 1.78609756       1.78 6103.90243 6130.24390 5777.56097     6120.0 6007.31707 92.0975609 
         3 16839.3958 17.7804878       22.0 30.8170731 31.8902439       21.0       12.0       5.13        3.4 1.78707317       1.78 6095.12195 6140.48780 5786.34146     6120.0 6004.39024 92.1463414 
         4 16839.4166 17.4812500 21.9875000 30.6125000    31.4375 20.6125000       12.0       5.13        3.4   1.785875       1.78     6111.0     6154.5     5778.0     6120.0     6003.0      93.35 
         5 16839.4375 17.4197530 21.5185185 30.6296296 31.4074074       20.0       12.0       5.13        3.4 1.78839506       1.78 6102.22222 6134.81481 5779.25925     6120.0     6000.0 94.5185185 
         6 16839.4583 17.2562500 21.1125000 30.2750000 31.1500000       20.0       12.0       5.13        3.4   1.788375       1.78     6102.0     6124.5     5779.5     6120.0     6003.0       92.0 
         7 16839.4791 16.9259259       21.0 30.0740740 31.1604938       20.0       12.0       5.13        3.4 1.78740740       1.78 6105.18518 6122.96296 5785.18518     6120.0 6013.33333 92.0493827 
         8    16839.5 16.8963414       21.0 30.0609756 31.0121951       20.0       12.0       5.13        3.4 1.78804878       1.78 6105.36585 6133.17073 5805.36585     6120.0 6005.85365 92.0487804 
         9 16839.5208 16.4329268 20.8170731 30.0121951 30.7560975 19.4390243       12.0       5.13        3.4 1.78878048       1.78 6106.82926 6137.56097 5768.78048     6120.0 6002.92682 92.0487804 
        14  16839.625 16.3475609 20.4634146 30.0121951 30.4756097       19.0       12.0       5.13        3.4 1.78902439       1.78 6102.43902 6130.24390 5787.80487     6120.0 6005.85365 92.0487804 
        18 16839.7083 16.6582278       21.0 30.0759493 30.5696202       20.0       12.0       5.13        3.4 1.78810126       1.78 6100.25316 6127.59493 5784.30379     6120.0 6007.59493 92.0506329 
        19 16839.7291 17.0379746 21.1772151 30.4936708 30.9620253       20.0       12.0       5.13        3.4 1.78772151       1.78 6095.69620 6123.03797 5772.15189     6120.0 6006.07594 92.0506329 
        20   16839.75 17.3961038 21.9740259 30.6493506 31.3766233 20.1038961       12.0       5.13        3.4 1.78844155       1.78 6101.29870 6121.55844 5777.14285     6120.0 6007.79220 92.0519480 
        21 16839.7708 17.7266666       22.0 30.8400000 31.7333333       21.0       12.0       5.13        3.4 1.78733333       1.78     6104.0     6132.8     5779.2     6120.0     6004.8 92.0533333 
       ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...  

In [12]:
plot = Nyaplot::Plot.new
plot.add_with_df(filtered_df, :scatter, :cpu1_1_8v, :cpu1_temp)
plot.x_label("CPU1 Voltage (V)")
plot.y_label("CPU1 temperature (℃)")
plot.show

#<Nyaplot::Frame:0x007f953c8d37d0 @properties={:panes=>[#<Nyaplot::Plot:0x007f953c8d6c78 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f953c8d69a8 @properties={:type=>:scatter, :options=>{:x=>:cpu1_1_8v, :y=>:cpu1_temp}, :data=>"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"}, @xrange=[1.78, 1.7895890410958906], @yrange=[30.012195121951265, 44.3648648648649]>], :options=>{:x_label=>"CPU1 Voltage (V)", :y_label=>"CPU1 temperature (℃)", :zoom=>true, :width=>700, :xrange=>[1.78, 1.7895890410958906], :yrange=>[30.012195121951265, 44.3648648648649]}}>], :data=>{"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"=>
#<Daru::DataFrame:70139492567880 @name = 8f45396d-a66c-4e4b-ac80-f4a63b36a9f8 @size = 266>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 16839.3333 18.8875000 23.0250000 32.2000000 32.7000000       22.0       12.0       5.13        3.4     1.7845       1.78     6102.0     6153.0     5775.0     6120.0     6006.0      92.15 
         1 16839.3541 18.5740740       23.0 32.0864197 32.3827160       22.0       12.0       5.13        3.4 1.78580246       1.78 6099.25925 6154.07407 5783.70370     6120.0 6004.44444       92.0 
         2  16839.375 18.1280487 22.3414634 31.2804878 32.2195121 21.0243902       12.0       5.13        3.4 1.78609756       1.78 6103.90243 6130.24390 5777.56097     6120.0 6007.31707 92.0975609 
         3 16839.3958 17.7804878       22.0 30.8170731 31.8902439       21.0       12.0       5.13        3.4 1.78707317       1.78 6095.12195 6140.48780 5786.34146     6120.0 6004.39024 92.1463414 
         4 16839.4166 17.4812500 21.9875000 30.6125000    31.4375 20.6125000       12.0       5.13        3.4   1.785875       1.78     6111.0     6154.5     5778.0     6120.0     6003.0      93.35 
         5 16839.4375 17.4197530 21.5185185 30.6296296 31.4074074       20.0       12.0       5.13        3.4 1.78839506       1.78 6102.22222 6134.81481 5779.25925     6120.0     6000.0 94.5185185 
         6 16839.4583 17.2562500 21.1125000 30.2750000 31.1500000       20.0       12.0       5.13        3.4   1.788375       1.78     6102.0     6124.5     5779.5     6120.0     6003.0       92.0 
         7 16839.4791 16.9259259       21.0 30.0740740 31.1604938       20.0       12.0       5.13        3.4 1.78740740       1.78 6105.18518 6122.96296 5785.18518     6120.0 6013.33333 92.0493827 
         8    16839.5 16.8963414       21.0 30.0609756 31.0121951       20.0       12.0       5.13        3.4 1.78804878       1.78 6105.36585 6133.17073 5805.36585     6120.0 6005.85365 92.0487804 
         9 16839.5208 16.4329268 20.8170731 30.0121951 30.7560975 19.4390243       12.0       5.13        3.4 1.78878048       1.78 6106.82926 6137.56097 5768.78048     6120.0 6002.92682 92.0487804 
        14  16839.625 16.3475609 20.4634146 30.0121951 30.4756097       19.0       12.0       5.13        3.4 1.78902439       1.78 6102.43902 6130.24390 5787.80487     6120.0 6005.85365 92.0487804 
        18 16839.7083 16.6582278       21.0 30.0759493 30.5696202       20.0       12.0       5.13        3.4 1.78810126       1.78 6100.25316 6127.59493 5784.30379     6120.0 6007.59493 92.0506329 
        19 16839.7291 17.0379746 21.1772151 30.4936708 30.9620253       20.0       12.0       5.13        3.4 1.78772151       1.78 6095.69620 6123.03797 5772.15189     6120.0 6006.07594 92.0506329 
        20   16839.75 17.3961038 21.9740259 30.6493506 31.3766233 20.1038961       12.0       5.13        3.4 1.78844155       1.78 6101.29870 6121.55844 5777.14285     6120.0 6007.79220 92.0519480 
        21 16839.7708 17.7266666       22.0 30.8400000 31.7333333       21.0       12.0       5.13        3.4 1.78733333       1.78     6104.0     6132.8     5779.2     6120.0     6004.8 92.0533333 
       ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...       

In [13]:
plot = Nyaplot::Plot.new
plot.add_with_df(filtered_df, :line, :time, :total_power)
plot.x_label("Time (days since epoch)")
plot.y_label("Total power (W)")
plot.show

#<Nyaplot::Frame:0x007f953f1060e0 @properties={:panes=>[#<Nyaplot::Plot:0x007f953f106b80 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f953f1069c8 @properties={:type=>:line, :options=>{:x=>:time, :y=>:total_power}, :data=>"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"}, @xrange=[16839.333333333332, 16846.3125], @yrange=[92.0, 107.89189189189189]>], :options=>{:x_label=>"Time (days since epoch)", :y_label=>"Total power (W)", :zoom=>true, :width=>700, :xrange=>[16839.333333333332, 16846.3125], :yrange=>[92.0, 107.89189189189189]}}>], :data=>{"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"=>
#<Daru::DataFrame:70139492567880 @name = 8f45396d-a66c-4e4b-ac80-f4a63b36a9f8 @size = 266>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 16839.3333 18.8875000 23.0250000 32.2000000 32.7000000       22.0       12.0       5.13        3.4     1.7845       1.78     6102.0     6153.0     5775.0     6120.0     6006.0      92.15 
         1 16839.3541 18.5740740       23.0 32.0864197 32.3827160       22.0       12.0       5.13        3.4 1.78580246       1.78 6099.25925 6154.07407 5783.70370     6120.0 6004.44444       92.0 
         2  16839.375 18.1280487 22.3414634 31.2804878 32.2195121 21.0243902       12.0       5.13        3.4 1.78609756       1.78 6103.90243 6130.24390 5777.56097     6120.0 6007.31707 92.0975609 
         3 16839.3958 17.7804878       22.0 30.8170731 31.8902439       21.0       12.0       5.13        3.4 1.78707317       1.78 6095.12195 6140.48780 5786.34146     6120.0 6004.39024 92.1463414 
         4 16839.4166 17.4812500 21.9875000 30.6125000    31.4375 20.6125000       12.0       5.13        3.4   1.785875       1.78     6111.0     6154.5     5778.0     6120.0     6003.0      93.35 
         5 16839.4375 17.4197530 21.5185185 30.6296296 31.4074074       20.0       12.0       5.13        3.4 1.78839506       1.78 6102.22222 6134.81481 5779.25925     6120.0     6000.0 94.5185185 
         6 16839.4583 17.2562500 21.1125000 30.2750000 31.1500000       20.0       12.0       5.13        3.4   1.788375       1.78     6102.0     6124.5     5779.5     6120.0     6003.0       92.0 
         7 16839.4791 16.9259259       21.0 30.0740740 31.1604938       20.0       12.0       5.13        3.4 1.78740740       1.78 6105.18518 6122.96296 5785.18518     6120.0 6013.33333 92.0493827 
         8    16839.5 16.8963414       21.0 30.0609756 31.0121951       20.0       12.0       5.13        3.4 1.78804878       1.78 6105.36585 6133.17073 5805.36585     6120.0 6005.85365 92.0487804 
         9 16839.5208 16.4329268 20.8170731 30.0121951 30.7560975 19.4390243       12.0       5.13        3.4 1.78878048       1.78 6106.82926 6137.56097 5768.78048     6120.0 6002.92682 92.0487804 
        14  16839.625 16.3475609 20.4634146 30.0121951 30.4756097       19.0       12.0       5.13        3.4 1.78902439       1.78 6102.43902 6130.24390 5787.80487     6120.0 6005.85365 92.0487804 
        18 16839.7083 16.6582278       21.0 30.0759493 30.5696202       20.0       12.0       5.13        3.4 1.78810126       1.78 6100.25316 6127.59493 5784.30379     6120.0 6007.59493 92.0506329 
        19 16839.7291 17.0379746 21.1772151 30.4936708 30.9620253       20.0       12.0       5.13        3.4 1.78772151       1.78 6095.69620 6123.03797 5772.15189     6120.0 6006.07594 92.0506329 
        20   16839.75 17.3961038 21.9740259 30.6493506 31.3766233 20.1038961       12.0       5.13        3.4 1.78844155       1.78 6101.29870 6121.55844 5777.14285     6120.0 6007.79220 92.0519480 
        21 16839.7708 17.7266666       22.0 30.8400000 31.7333333       21.0       12.0       5.13        3.4 1.78733333       1.78     6104.0     6132.8     5779.2     6120.0     6004.8 92.0533333 
       ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ... 

In [14]:
plot = Nyaplot::Plot.new
plot.add_with_df(filtered_df, :line, :time, :fan1_sys)
plot.x_label("Time (days since epoch)")
plot.y_label("Fan1 (rpm)")
plot.show

#<Nyaplot::Frame:0x007f953dcf9570 @properties={:panes=>[#<Nyaplot::Plot:0x007f953dcfa038 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f953dcf9e80 @properties={:type=>:line, :options=>{:x=>:time, :y=>:fan1_sys}, :data=>"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"}, @xrange=[16839.333333333332, 16846.3125], @yrange=[6083.333333333333, 9360.0]>], :options=>{:x_label=>"Time (days since epoch)", :y_label=>"Fan1 (rpm)", :zoom=>true, :width=>700, :xrange=>[16839.333333333332, 16846.3125], :yrange=>[6083.333333333333, 9360.0]}}>], :data=>{"8f45396d-a66c-4e4b-ac80-f4a63b36a9f8"=>
#<Daru::DataFrame:70139492567880 @name = 8f45396d-a66c-4e4b-ac80-f4a63b36a9f8 @size = 266>
                 time ambient_te systemboar  cpu1_temp  cpu2_temp dimm_1a_te   main_12v    main_5v  main_3_3v  cpu1_1_8v  cpu2_1_8v   fan1_sys   fan2_sys   fan3_sys   fan4_sys   fan5_sys total_powe 
         0 16839.3333 18.8875000 23.0250000 32.2000000 32.7000000       22.0       12.0       5.13        3.4     1.7845       1.78     6102.0     6153.0     5775.0     6120.0     6006.0      92.15 
         1 16839.3541 18.5740740       23.0 32.0864197 32.3827160       22.0       12.0       5.13        3.4 1.78580246       1.78 6099.25925 6154.07407 5783.70370     6120.0 6004.44444       92.0 
         2  16839.375 18.1280487 22.3414634 31.2804878 32.2195121 21.0243902       12.0       5.13        3.4 1.78609756       1.78 6103.90243 6130.24390 5777.56097     6120.0 6007.31707 92.0975609 
         3 16839.3958 17.7804878       22.0 30.8170731 31.8902439       21.0       12.0       5.13        3.4 1.78707317       1.78 6095.12195 6140.48780 5786.34146     6120.0 6004.39024 92.1463414 
         4 16839.4166 17.4812500 21.9875000 30.6125000    31.4375 20.6125000       12.0       5.13        3.4   1.785875       1.78     6111.0     6154.5     5778.0     6120.0     6003.0      93.35 
         5 16839.4375 17.4197530 21.5185185 30.6296296 31.4074074       20.0       12.0       5.13        3.4 1.78839506       1.78 6102.22222 6134.81481 5779.25925     6120.0     6000.0 94.5185185 
         6 16839.4583 17.2562500 21.1125000 30.2750000 31.1500000       20.0       12.0       5.13        3.4   1.788375       1.78     6102.0     6124.5     5779.5     6120.0     6003.0       92.0 
         7 16839.4791 16.9259259       21.0 30.0740740 31.1604938       20.0       12.0       5.13        3.4 1.78740740       1.78 6105.18518 6122.96296 5785.18518     6120.0 6013.33333 92.0493827 
         8    16839.5 16.8963414       21.0 30.0609756 31.0121951       20.0       12.0       5.13        3.4 1.78804878       1.78 6105.36585 6133.17073 5805.36585     6120.0 6005.85365 92.0487804 
         9 16839.5208 16.4329268 20.8170731 30.0121951 30.7560975 19.4390243       12.0       5.13        3.4 1.78878048       1.78 6106.82926 6137.56097 5768.78048     6120.0 6002.92682 92.0487804 
        14  16839.625 16.3475609 20.4634146 30.0121951 30.4756097       19.0       12.0       5.13        3.4 1.78902439       1.78 6102.43902 6130.24390 5787.80487     6120.0 6005.85365 92.0487804 
        18 16839.7083 16.6582278       21.0 30.0759493 30.5696202       20.0       12.0       5.13        3.4 1.78810126       1.78 6100.25316 6127.59493 5784.30379     6120.0 6007.59493 92.0506329 
        19 16839.7291 17.0379746 21.1772151 30.4936708 30.9620253       20.0       12.0       5.13        3.4 1.78772151       1.78 6095.69620 6123.03797 5772.15189     6120.0 6006.07594 92.0506329 
        20   16839.75 17.3961038 21.9740259 30.6493506 31.3766233 20.1038961       12.0       5.13        3.4 1.78844155       1.78 6101.29870 6121.55844 5777.14285     6120.0 6007.79220 92.0519480 
        21 16839.7708 17.7266666       22.0 30.8400000 31.7333333       21.0       12.0       5.13        3.4 1.78733333       1.78     6104.0     6132.8     5779.2     6120.0     6004.8 92.0533333 
       ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...       